In [1]:
#%pip install -U transformers

In [2]:
import transformers; 
print(transformers.__version__)


4.57.3


In [ ]:
#%pip install -U git+https://github.com/huggingface/diffusers

In [2]:
#%pip install accelerate

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
if torch.cuda.is_available():
    torch_dtype = torch.bfloat16
    device = "cuda"
else:
    torch_dtype = torch.float32
    device = "cpu"

print(f"device: {device}")

device: cuda


In [1]:
import os
from diffusers import QwenImagePipeline
import torch
# Load the pipeline
if torch.cuda.is_available():
    torch_dtype = torch.bfloat16
    device = "cuda"
else:
    torch_dtype = torch.float32
    device = "cpu"

print(f"device: {device}")


# Optional but strongly recommended
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

pipe = QwenImagePipeline.from_pretrained(
    "./qwen_image_get_model",
    torch_dtype=torch.bfloat16,
    device_map="balanced",
    max_memory={
        0: "38GiB",    # GPU headroom (critical)
        "cpu": "64GiB"
    },
    local_files_only=True
)

#pipe.enable_attention_slicing()
#pipe.enable_xformers_memory_efficient_attention()



# Generate image
prompt = '''A 20-year-old East Asian girl with delicate, charming features and large, bright brown eyes—expressive and lively, with a cheerful or subtly smiling expression. Her naturally wavy long hair is either loose or tied in twin ponytails. She has fair skin and light makeup accentuating her youthful freshness. She wears a modern, cute dress or relaxed outfit in bright, soft colors—lightweight fabric, minimalist cut. She stands indoors at an anime convention, surrounded by banners, posters, or stalls. Lighting is typical indoor illumination—no staged lighting—and the image resembles a casual iPhone snapshot: unpretentious composition, yet brimming with vivid, fresh, youthful charm.'''

negative_prompt = "低分辨率，低画质，肢体畸形，手指畸形，画面过饱和，蜡像感，人脸无细节，过度光滑，画面具有AI感。构图混乱。文字模糊，扭曲。"


# Generate with different aspect ratios
aspect_ratios = {
    "1:1": (1328, 1328),
    "16:9": (1664, 928),
    "9:16": (928, 1664),
    "4:3": (1472, 1104),
    "3:4": (1104, 1472),
    "3:2": (1584, 1056),
    "2:3": (1056, 1584),
}

width, height = aspect_ratios["16:9"]

image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=width,
    height=height,
    num_inference_steps=10, #50
    true_cfg_scale=4.0,
    generator=torch.Generator(device="cuda").manual_seed(42)
).images[0]

image.save("example.png")


device: cuda


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


  0%|          | 0/10 [00:00<?, ?it/s]